# 2023 AI/ML UnicornGym - Track 1

@jesamkim | 2023-Feb-08

# 9. 전체 모델 빌딩 파이프라인 개발
데이터 전처리 스텝은 제외되었으며, 나머지 훈련, 모델, 배포에 대한 전체 파이프라인을 생성 합니다.

### setup environment

In [1]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

## 9-1. SageMaker 모델 빌드 파이프라인을 이용한 모델 빌드 오케스트레이션

Amazon SageMaker 모델 구축 파이프라인은 직접 SageMaker 통합을 활용하는 머신 러닝 파이프라인을 구축하기 위한 도구입니다. 

- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)

## 9-2. 파이프라인 개발자 가이드
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)

![mdp_how_it_works.png](img/mdp_how_it_works.png)

SageMaker 파이프라인은 다음 기능을 지원하며 본 노트북에서 하나씩 다루게 됩니다. 

* Processing job steps - 데이터처러 워크로드를 실행하기 위한 SageMaker의 관리형 기능. Feature engineering, 데이터 검증, 모델 평가, 모델 해석 등에 주로 사용됨 
* Training job steps - 학습작업. 모델에게 학습데이터셋을 이용하여 모델에게 예측을 하도록 학습시키는 반복적인 작업 
* Create model steps - 추론 엔드포인트 또는 배치 추론을 위한 모델의 생성 
* Pipelines - Workflow DAG. SageMaker 작업과 리소스 생성을 조율하는 단계와 조건을 가진다. 

## 9-3. 기본 라이브러리 로딩 

- [중요] Sagemaker python sdk 를 `2.72.1` 로 다운 그레이드 합니다. 현재는 2.77.x 버전이지만 CreateModel 단계의 오류로 인해서 다운 그레이드 합니다.
- 세이지 메이커 관련 라이브러리를 로딩 합니다.

In [2]:
import sys, IPython

sm_sdk_version = "2.72.1"

import sagemaker
current_sm_version = sagemaker.__version__
if sm_sdk_version == current_sm_version:
    print("current_sm_version: ", current_sm_version)
else:
    !{sys.executable} -m pip install sagemaker=={sm_sdk_version} --upgrade    
    IPython.Application.instance().kernel.do_shutdown(True)    


current_sm_version:  2.72.1


In [3]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

### 노트북 변수 로딩

In [4]:
%store -r

%store

Stored variables and their in-db values:
bucket                           -> 'sagemaker-us-east-1-376278017302'
endpoint_name                    -> 'sagemaker-xgboost-2023-02-09-05-24-37-494'
hyperparameters                  -> {'scale_pos_weight': '19', 'max_depth': '2', 'eta'
image_uri                        -> '683313688378.dkr.ecr.us-east-1.amazonaws.com/sage
input_data_uri                   -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
project_prefix                   -> 'titanic-ml'
s3_path                          -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
sagemaker_model                  -> 'pipelines-4jksbfxkgkny-survived-basic-creat-km39j
test_data_uri                    -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
test_df                          ->     PassengerId  Pclass  Sex   Age  SibSp  Parch  
train_data_uri                   -> 's3://sagemaker-us-east-1-376278017302/titanic-ml/
train_model_artifact             -> 's3://sagemaker-us-east-1-3

---
# 9-4. 모델 빌딩 파이프라인 의 스텝(Step) 생성

### 모델 빌딩 파이프라인 변수 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 프로세싱 스텝을 위한 인스턴스 타입 및 인스턴스 수
    - 데이터 전처리 스텝 및 실시간 앤드 포인트 스텝에 사용 됨.
- 훈련 스텝을 위한 인스턴스 타입 및 인스턴스 수     
- 엔트 포인트의 인스턴스 타입
- 원본 데이터 세트에 대한 S3 주소
    - 데이터 전처리 스텝에서 사용 됩니다.

In [5]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)

endpoint_instance_type = ParameterString(
    name="EndpointInstanceType",
    default_value="ml.m5.xlarge"
)

input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri
)

### 모델 학습을 위한 학습단계 정의


학습 스텝을 정의하기 위해서는 크게 아래와 같은 과정이 있습니다.
- XGBoost Estimator 정의
- 학습 스텝 정의
    - 아래와 같은 중요한 인자가 필요 합니다.
        - Estimator (위에서 정의한 것 사용)
        - 훈련을 위한 입력 데이터 위치

### 기본 훈련 변수 및 하이퍼파라미터 설정

In [6]:
from sagemaker.xgboost.estimator import XGBoost

estimator_output_path = f's3://{bucket}/{project_prefix}/training-jobs'
XGB_FRAMEWORK_VERSION = '1.3-1'


hyperparameters = {
       "scale_pos_weight" : "19",    
        "max_depth": "2",
        "eta": "0.3",
        "objective": "binary:logistic",
        "num_round": "100",
}

In [7]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script_pipeline.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version=XGB_FRAMEWORK_VERSION    
)

훈련의 입력은 S3 출력 경로를 지정하고, 다음 스텝부터 이전의 S3 경로 결과가 제공됩니다.
- `step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri`

In [8]:
train_preproc_dir_artifact = input_data_uri 

In [9]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="Titanic-Basic-Train",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data = train_preproc_dir_artifact,
            content_type="text/csv"
        ),
    },
)

### SageMaker 모델 생성 스텝 생성
- 스텝 생성시에 위에서 생성한 Estimator 입력 및 입력 데이타로서 이전에 생성한 S3 경로를 제공합니다.

In [10]:
from sagemaker.model import Model

model = Model(
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [11]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
)
step_create_model = CreateModelStep(
    name="Titanic-Basic-CreateModel",
    model=model,
    inputs=inputs,
)

### 실시간 엔드 포인트 배포 스텝 생성

앤드포인트를 생성하기 위해서는 프로세싱 스텝을 통해서 합니다. 프레세싱 스텝에 앤드포인트 생성에 필요한 코드(스크립트)를 작성하여 프로세싱 스텝에서 실행하여 생성하게 합니다. 크게 아래와 같은 과정으로 합니다.

- 앤드포인트 생성 코드를 S3 에 업로드
- SKLearnProcessor 오브젝트 생성
- ProcessingStep 정의 (중요한 인자는 아래와 같습니다.)
    - processor (SKLearnProcessor 오브젝트 제공)
    - 코드에 전달할 커맨드 인자
        - endpoint config 생성시에, 이전 단계의 모델 결과를 제공합니다.
        - "--model_name", step_create_model.properties.ModelName,     
    - 앤드포인트 생성 코드



### 엔드포인트 생성 스크립트 코드를 S3에 업로딩

In [12]:
from datetime import datetime
suffix = datetime.now().microsecond

In [13]:

local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: ", s3_deploy_code_uri)

all_pipeline_endpoint_name = 'merge-pipeline-endpoint-' + str(suffix)
endpoint_instance_type = "ml.m5.xlarge"

s3_deploy_code_uri:  s3://sagemaker-us-east-1-376278017302/titanic-ml/code/deploy_model.py


### 프로세서 정의 및 스텝 정의

In [14]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role= role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name='Survived-Basic-Endpoint',
    sagemaker_session=sagemaker_session)


step_deploy = ProcessingStep(
    name='Titanic-Basic-Endpoint',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", step_create_model.properties.ModelName, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", all_pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

## 9-5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행


이제 지금까지 생성한 단계들을 하나의 파이프라인으로 조합하고 실행하도록 하겠습니다.

파이프라인은 name, parameters, steps 속성이 필수적으로 필요합니다. 
여기서 파이프라인의 이름은 (account, region) 조합에 대하여 유일(unique))해야 합니다.
우리는 또한 여기서 Experiment 설정을 추가 하여, 실험에 등록 합니다.

주의:

- 정의에 사용한 모든 파라미터가 존재해야 합니다.
- 파이프라인으로 전달된 단계(step)들은 실행순서와는 무관합니다. SageMaker Pipeline은 단계가 실행되고 완료될 수 있도록 의존관계를를 해석합니다.

### 파이프라인 정의
- 위에서 정의한 파라미터를 제공
- 실행할 스텝 기술
    - steps=[step_train, step_create_model, step_deploy],
- 아래는 약 10분 정도 소요 됩니다.

In [15]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        training_instance_count,   
        #endpoint_instance_type,        
        input_data,
    ],
   steps=[step_train, step_create_model, step_deploy],

)

### 파이프라인 정의 확인

위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [16]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'TrainingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-376278017302/titanic-ml/input'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Titanic-Basic-Train',
   'Type': 'Training',
   'Arguments': {'AlgorithmSpecification': {'TrainingInputMode': 'File',
     'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1'},
    'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-376278017302/titanic-ml/training-jobs'}

### 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [17]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

디폴트값을 이용하여 파이프라인을 샐행합니다.


In [18]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:376278017302:pipeline/titanic-ml/execution/jr5f6kq7io1e',
 'PipelineExecutionDisplayName': 'execution-1675921599005',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2023, 2, 9, 5, 46, 38, 898000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 2, 9, 5, 46, 38, 898000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '5b7e4d95-a6a4-4d17-8abf-395b94039096',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5b7e4d95-a6a4-4d17-8abf-395b94039096',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '385',
   'date': 'Thu, 09 Feb 2023 05:46:38 GMT'},
  'RetryAttempts': 0}}

### 파이프라인 실행 기다리기

In [19]:
execution.wait()

실행이 완료될 때까지 기다립니다.


### 파이프라인 실행 단계 기록 보기

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.


In [20]:
execution.list_steps()

[{'StepName': 'Titanic-Basic-Endpoint',
  'StartTime': datetime.datetime(2023, 2, 9, 5, 49, 18, 124000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 2, 9, 5, 55, 59, 195000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:376278017302:processing-job/pipelines-jr5f6kq7io1e-titanic-basic-endpoi-plrlnzzl7t'}}},
 {'StepName': 'Titanic-Basic-CreateModel',
  'StartTime': datetime.datetime(2023, 2, 9, 5, 49, 16, 622000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 2, 9, 5, 49, 17, 657000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:376278017302:model/pipelines-jr5f6kq7io1e-titanic-basic-create-hi9zrltiri'}}},
 {'StepName': 'Titanic-Basic-Train',
  'StartTime': datetime.datetime(2023, 2, 9, 5, 46, 39, 978000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 2, 9, 5, 49, 15, 929000, tzinfo=tzl

---
## 9-6. 세이지 메이커 스튜디오에서 실행 확인 하기

- 아래의 그림 처럼 SageMaker Studio에 로긴후에 따라하시면, SageMaker Studio 에서도 실행 내역을 확인할 수 있습니다.
- SageMaker Studio 개발자 가이드 --> [SageMaker Studio](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/studio.html)

![sm_pipeline_all.png](img/sm_pipeline_all.png)

---
## 9-7. 아티펙트 경로 추출
실제 추론시에 필요한 아래 두가지를 추출 합니다.
- 전처리된 테스트 데이터의 경로
- endpoint 이름을 추출

In [21]:
from src.p_utils import get_proc_artifact

import boto3
client = boto3.client("sagemaker")

#test_preproc_dir_artifact = get_proc_artifact(execution, client, kind=1 )

test_preproc_dir_artifact = input_data_uri

print("test_preproc__dir_artifact: ", test_preproc_dir_artifact)



test_preproc__dir_artifact:  s3://sagemaker-us-east-1-376278017302/titanic-ml/input


### 변수 저장

In [22]:
%store test_preproc_dir_artifact
%store all_pipeline_endpoint_name

Stored 'test_preproc_dir_artifact' (str)
Stored 'all_pipeline_endpoint_name' (str)
